<h1 align="center">DotNetStac</h1>
<h2 align="center">
.Net library for working with Spatio Temporal Asset Catalogs (<a href="https://stacspec.org">STAC</a>)
</h2>

### **DotNetStac** helps you to work with [STAC](https://stacspec.org) ([catalog](https://github.com/radiantearth/stac-spec/tree/master/catalog-spec), [collection](https://github.com/radiantearth/stac-spec/tree/master/collection-spec), [item](https://github.com/radiantearth/stac-spec/tree/master/catalog-spec))
In a nutshell, the library allows import/export of STAC JSON documents (Serialization/Deserialization using [Newtonsoft.JSON](https://www.newtonsoft.com/json)) to typed object with properties represented in enhanced objects such as geometries, time stamp/period/span, numerical values and many more via STAC extension plugins engine.

## Getting Started

In this notebook, we are going to review the library's features through a series of code examples.

#### Important Note about .Net programming and C# language usage in this library

We chose to design and implement the objects of this library as plain as possible (e.g. POCO) in order to keep the structured and typed nature of the C# language. So, we will deal with normal classes, without more attributes describing infrastructure concerns or other responsibilities that your domain objects shouldn't have.

### 1 Install DotNetStac
First, we either install lastest version of DotNetStac in the notebook or  we use the locally built binaries

In [1]:
// Use built lib
#r "/home/jovyan/src/DotNetStac/bin/Debug/DotNetStac.dll"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Restore sources /tmp/

Installed package DotNetStac version 0.7.1.1-alpha

### 2 Deserialization of STAC documents

The (de)serialiation methods are wrapped in methods in [class `StacConvert`](https://terradue.github.io/DotNetStac/api/Stac.StacConvert.html) that is the main entry point from/to JSON/.Net.

Let's start reading a STAC catalog online. Please note that DotNetStac does not provide with data access middleware. You can integrate own data access or you can test the [`Stars` SDK](https://github.com/Terradue/Stars) that provides with integrated functions to manipulate STAC objects and their storage.

The following code is a very simple function loading a catalog and printing it's `id`, `description` and `stac_version`.

In [7]:
using Stac;
using System;
using System.Net;

var webc = new WebClient();
Uri catalogUri = new Uri("https://storage.googleapis.com/earthengine-stac-experimental/catalog-v015/catalog.json");

// StacConvert.Deserialize is the helper to start loading any STAC document
var json = webc.DownloadString(catalogUri);
StacCatalog catalog = StacConvert.Deserialize<StacCatalog>(json);

Console.Out.WriteLine(catalog.Id + ": " + catalog.Description);
Console.Out.WriteLine(catalog.StacVersion);


GEE_catalog: The Engine Engine Public Data Catalog.

See also:

- https://developers.google.com/earth-engine/datasets/catalog

1.0.0-rc.2


### 3 Navigation from a Catalogue

Using the previously loaded catalog, the following code executes a recursive function navigating from a root catalog through all it's tree structure recursing in the `child` STAC links and listing the `item` links and their `assets`.

Please note the following:

- `GetChildrenLinks` and `GetItemLinks` are the recommanded ways to get the links for navigating through the tree.
- The previous functions as the rest of the library does not alter the `Uri`s. It is then up to the developer to resolve the relative ones. As in the code, Uri class provides with all the necessary methods to easily join a base Url with a relative one.
- The `StacConvert.Deserialize<>` methods allows to specify the interfaces to ease the deserialization when the STAC type is unknown: [`IStacObject`](https://terradue.github.io/DotNetStac/api/Stac.IStacObject.html) and [`IStacCatalog`](https://terradue.github.io/DotNetStac/api/Stac.IStacCatalog.html).

In [9]:
using System.Linq;
using Stac.Extensions.ItemCollections;

public static void ListChildrensItemsAndAssets(IStacParent catalog, Uri baseUri, WebClient webc, string prefix = "", int limit = 20)
{
    // Get children and items (sub catalogs, collections and items)
    foreach (var childLink in catalog.GetChildrenLinks().Concat(catalog.GetItemLinks()).Take(limit))
    {
        // IMPORTANT: Relative Uri resolution
        Uri childUri = childLink.Uri;
        if (!childUri.IsAbsoluteUri)
            childUri = new Uri(baseUri, childUri.ToString());
        var childjson = webc.DownloadString(childUri);
        // STAC object loading (using the IStacObject interface)
        IStacObject child = StacConvert.Deserialize<IStacObject>(childjson);

        if (child is StacCatalog || child is StacCollection)
            Console.Out.WriteLine(prefix + child.Id + ": " + child.CommonMetadata().Title);

        List<StacItem> items = new List<StacItem>();

        // Item or ItemCollection
        if (child is StacItem)
            items.Add(child as StacItem);
        if (child is ItemCollection)
            items = (child as ItemCollection).Features;
        // List assets if item
        foreach (var item in items)
        {
            // Print the item
            Console.Out.WriteLine(prefix + "  " + item.Id + ": " + item.CommonMetadata().Title);
            foreach (var asset in item.Assets.Values) {
                Console.Out.WriteLine(prefix + "    *[" + asset.MediaType + "] " + asset.Uri);
            }
        }

        // Go deeper if catalog or collection
        if (child is StacCatalog || child is StacCollection){
            ListChildrensItemsAndAssets(child as IStacParent, childUri, webc, prefix + "  ", limit);
        }
        
    }
}

// Start the navigation
ListChildrensItemsAndAssets(catalog as IStacParent, catalogUri, webc);

AAFC: 
  AAFC/ACI: 
AHN: 
  AHN/AHN2_05M_INT: 
  AHN/AHN2_05M_NON: 
  AHN/AHN2_05M_RUW: 
ASTER: 
  ASTER/AST_L1T_003: 
AU: 
  AU/GA/AUSTRALIA_5M_DEM: 
  AU/GA/DEM_1SEC/v10/DEM-H: 
  AU/GA/DEM_1SEC/v10/DEM-S: 
BIOPAMA: 
  BIOPAMA/GlobalOilPalm/v1: 
BLM: 
  BLM/AIM/v1/TerrADat/TerrestrialAIM: 
CAS: 
  CAS/IGSNRR/PML/V2: 
CGIAR: 
  CGIAR/SRTM90_V4: 
CIESIN: 
  CIESIN/GPWv411/GPW_Basic_Demographic_Characteristics: 
  CIESIN/GPWv411/GPW_Data_Context: 
  CIESIN/GPWv411/GPW_Land_Area: 
  CIESIN/GPWv411/GPW_Mean_Administrative_Unit_Area: 
  CIESIN/GPWv411/GPW_National_Identifier_Grid: 
  CIESIN/GPWv411/GPW_Population_Count: 
  CIESIN/GPWv411/GPW_Population_Density: 
  CIESIN/GPWv411/GPW_UNWPP-Adjusted_Population_Count: 
  CIESIN/GPWv411/GPW_UNWPP-Adjusted_Population_Density: 
  CIESIN/GPWv411/GPW_Water_Area: 
  CIESIN/GPWv411/GPW_Water_Mask: 
  CIESIN/GPWv4/ancillary-data-grids: 
  CIESIN/GPWv4/population-count: 
  CIESIN/GPWv4/population-density: 
  CIESIN/GPWv4/unwpp-adjusted-population-coun

### 4 Creation and serialization of STAC documents

#### 4.1 Collection creation from scratch

Let's now create a STAC collection programmatically using the plain Collection object. Here all collection specific fields are set manually.

In [8]:
using Newtonsoft.Json.Linq;
using Stac;
using Stac.Collection;
using System;
using System.Collections.Generic;


  // First the mandatory elements of a Collection
  // Spatial Extent
  StacExtent extent = new StacExtent();
  extent.Spatial = new StacSpatialExtent(-180, -56, 180, 83);
  // Temporal Extent (set null to specify an open time extent)
  extent.Temporal = new StacTemporalExtent(DateTime.Parse("2015-06-23T00:00:00Z").ToUniversalTime(), null);

  // Create the Collection
  StacCollection collection = new StacCollection("COPERNICUS/S2",
    "Sentinel-2 is a wide-swath, high-resolution, multi-spectral\nimaging mission supporting Copernicus Land Monitoring studies,...",
                                      extent);

  // Title
  collection.Title = "Sentinel-2 MSI: MultiSpectral Instrument, Level-1C";

  // Links (self, parent, root, other) via different helpers
  collection.Links.Add(StacLink.CreateSelfLink(new Uri("https://storage.cloud.google.com/earthengine-test/catalog/COPERNICUS_S2.json")));
  collection.Links.Add(StacLink.CreateParentLink(new Uri("https://storage.cloud.google.com/earthengine-test/catalog/catalog.json")));
  collection.Links.Add(StacLink.CreateRootLink(new Uri("https://storage.cloud.google.com/earthengine-test/catalog/catalog.json")));
  collection.Links.Add(new StacLink(new Uri("https://scihub.copernicus.eu/twiki/pub/SciHubWebPortal/TermsConditions/Sentinel_Data_Terms_and_Conditions.pdf"), "license", "Legal notice on the use of Copernicus Sentinel Data and Service Information", null));

  // Keywords
  collection.Keywords.Add("copernicus");
  collection.Keywords.Add("esa");
  collection.Keywords.Add("eu");
  collection.Keywords.Add("msi");
  collection.Keywords.Add("radiance");
  collection.Keywords.Add("sentinel");

  // Collection of Providers
  collection.Providers.Add(new StacProvider("European Union/ESA/Copernicus",
                            new List<StacProviderRole>() { StacProviderRole.producer, StacProviderRole.licensor })
            {
                Uri = new Uri("https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi")
            });

  // Summaries
  // Stat Summary for the dates
  collection.Summaries.Add("datetime",
      new StacSummaryStatsObject<DateTime>(
          DateTime.Parse("2015-06-23T00:00:00Z").ToUniversalTime(),
          DateTime.Parse("2019-07-10T13:44:56Z").ToUniversalTime()
      )
  );

  // Value Set Summary for the platforms
  collection.Summaries.Add("platform",
      new StacSummaryValueSet<string>(new string[] { "sentinel-2a", "sentinel-2b" })
  );

  collection.Summaries.Add("constellation",
      new StacSummaryValueSet<string>(new string[] { "sentinel-2" })
  );

  collection.Summaries.Add("instruments",
      new StacSummaryValueSet<string>(new string[] { "msi" })
  );

  // Stat Summary for specific extensions
  collection.Summaries.Add("view:off_nadir",
      new StacSummaryStatsObject<double>(
          0.0,
          100
      )
  );

  collection.Summaries.Add("view:sun_elevation",
      new StacSummaryStatsObject<double>(
          6.78,
          89.9
      )
  );

  collection.Summaries.Add("sci:citation",
      new StacSummaryValueSet<string>(new string[] { "Copernicus Sentinel data [Year]" })
  );

  collection.Summaries.Add("gsd",
      new StacSummaryValueSet<int>(new int[] {
          10,
          30,
          60
      })
  );

  collection.Summaries.Add("proj:epsg",
      new StacSummaryValueSet<int>(new int[]
          { 32601,32602,32603,32604,32605,32606,32607,32608,32609,32610,32611,32612,32613,32614,32615,32616,32617,32618,32619,32620,32621,32622,32623,32624,32625,32626,32627,32628,32629,32630,32631,32632,32633,32634,32635,32636,32637,32638,32639,32640,32641,32642,32643,32644,32645,32646,32647,32648,32649,32650,32651,32652,32653,32654,32655,32656,32657,32658,32659,32660}
      )
  );

  // Value Set Summary can also be set using JToken objects from Newtonsoft.Json library
  collection.Summaries.Add("eo:bands",
      new StacSummaryValueSet<JObject>(new JObject[] {
          new JObject {
              { "name", "B1" },
              { "common_name", "coastal" },
              { "center_wavelength", 4.439 }
          },
          new JObject {
              { "name", "B2"},
              { "common_name", "blue"},
              { "center_wavelength", 4.966}
          },
          new JObject {
              { "name", "B3"},
              { "common_name", "green"},
              { "center_wavelength", 5.6}
          },
          new JObject {
              { "name", "B4"},
              { "common_name", "red"},
              { "center_wavelength", 6.645}
          },
          new JObject {
              { "name", "B5"},
              { "center_wavelength", 7.039}
          },
          new JObject {
              { "name", "B6"},
              { "center_wavelength", 7.402}
          },
          new JObject {
              { "name", "B7"},
              { "center_wavelength", 7.825}
          },
          new JObject {
              { "name", "B8"},
              { "common_name", "nir"},
              { "center_wavelength", 8.351}
          },
          new JObject {
              { "name", "B8A"},
              { "center_wavelength", 8.648}
          },
          new JObject {
              { "name", "B9"},
              { "center_wavelength", 9.45}
          },
          new JObject {
              { "name", "B10"},
              { "center_wavelength", 1.3735}
          },
          new JObject {
              { "name", "B11"},
              { "common_name", "swir16"},
              { "center_wavelength", 1.6137}
          },
          new JObject {
              { "name", "B12"},
              { "common_name", "swir22"},
              { "center_wavelength", 2.2024}
          }
      })
  );

  // Serialize
  var json = StacConvert.Serialize(collection);
  // Print JSON!
  Console.WriteLine(json)

{"id":"COPERNICUS/S2","stac_version":"1.0.0-rc.2","links":[{"rel":"self","href":"https://storage.cloud.google.com/earthengine-test/catalog/COPERNICUS_S2.json"},{"rel":"parent","href":"https://storage.cloud.google.com/earthengine-test/catalog/catalog.json"},{"rel":"root","href":"https://storage.cloud.google.com/earthengine-test/catalog/catalog.json"},{"rel":"license","title":"Legal notice on the use of Copernicus Sentinel Data and Service Information","href":"https://scihub.copernicus.eu/twiki/pub/SciHubWebPortal/TermsConditions/Sentinel_Data_Terms_and_Conditions.pdf"}],"type":"Collection","description":"Sentinel-2 is a wide-swath, high-resolution, multi-spectral\nimaging mission supporting Copernicus Land Monitoring studies,...","title":"Sentinel-2 MSI: MultiSpectral Instrument, Level-1C","summaries":{"datetime":{"minimum":"2015-06-23T00:00:00Z","maximum":"2019-07-10T13:44:56Z"},"platform":["sentinel-2a","sentinel-2b"],"constellation":["sentinel-2"],"instruments":["msi"],"view:off_nadi

#### 4.2 Collection generation from a set of Items

[`StacCollection` class](https://terradue.github.io/DotNetStac/api/Stac.StacCollection.html) has static methods allowing the automatic generation of a collection from a set of `StacItem`. The following code loads the items of [the examples folder from STAC repository](https://github.com/radiantearth/stac-spec/tree/master/examples) and generates the corresponding collection with
- **Spatial and temporal extent** from geometry and time merge of the items
- **Fields summaries** with stats objects or value sets of the items' fields values
  
Please note that the function takes also the eventual uri of the collection in input. If specified, the items Uri are made relative to that uri.

In [11]:
Uri simpleItemUri = new Uri("https://raw.githubusercontent.com/radiantearth/stac-spec/dev/examples/simple-item.json");
Uri coreItemUri = new Uri("https://raw.githubusercontent.com/radiantearth/stac-spec/dev/examples/core-item.json");
Uri extendedItemUri = new Uri("https://raw.githubusercontent.com/radiantearth/stac-spec/dev/examples/extended-item.json");

StacItem simpleItem = StacConvert.Deserialize<StacItem>(webc.DownloadString(simpleItemUri));
StacItem coreItem = StacConvert.Deserialize<StacItem>(webc.DownloadString(coreItemUri));
StacItem extendedItem = StacConvert.Deserialize<StacItem>(webc.DownloadString(extendedItemUri));

Dictionary<Uri, StacItem> items = new Dictionary<Uri, StacItem>();
items.Add(simpleItemUri, simpleItem);
items.Add(coreItemUri,coreItem);
items.Add(extendedItemUri, extendedItem);

StacCollection stacCollection = StacCollection.Create(new Uri($"https://raw.githubusercontent.com/radiantearth/stac-spec/dev/examples/collection.json"),
                                                                "simple-collection",
                                                                "A simple collection demonstrating core catalog fields with links to a couple of items",
                                                                items,
                                                                null,
                                                                "CC-BY-4.0");

Console.Out.Write(StacConvert.Serialize(collection));



{"id":"COPERNICUS/S2","stac_version":"1.0.0-rc.2","links":[{"rel":"self","href":"https://storage.cloud.google.com/earthengine-test/catalog/COPERNICUS_S2.json"},{"rel":"parent","href":"https://storage.cloud.google.com/earthengine-test/catalog/catalog.json"},{"rel":"root","href":"https://storage.cloud.google.com/earthengine-test/catalog/catalog.json"},{"rel":"license","title":"Legal notice on the use of Copernicus Sentinel Data and Service Information","href":"https://scihub.copernicus.eu/twiki/pub/SciHubWebPortal/TermsConditions/Sentinel_Data_Terms_and_Conditions.pdf"}],"type":"Collection","description":"Sentinel-2 is a wide-swath, high-resolution, multi-spectral\nimaging mission supporting Copernicus Land Monitoring studies,...","title":"Sentinel-2 MSI: MultiSpectral Instrument, Level-1C","summaries":{"datetime":{"minimum":"2015-06-23T00:00:00Z","maximum":"2019-07-10T13:44:56Z"},"platform":["sentinel-2a","sentinel-2b"],"constellation":["sentinel-2"],"instruments":["msi"],"view:off_nadi

### 5 STAC Extensions support

The library implements STAC extensions as extensions classes for enabling quick fields accessors. A extension class may also implement helpers related to the extensions such as 
- projection: WKT generation from EPSG identifier
- sar: interferometric search
- raster: calibration parameters

In [5]:
using System.IO;
using Stac.Extensions.Sat;

// Local File loading
var s1_1_filepath = Path.GetFullPath("files/S1A_IW_SLC__1SDV_20150305T051937_20150305T052005_004892_006196_ABBB.json", Directory.GetCurrentDirectory());
var s1_2_filepath = Path.GetFullPath("files/S1A_IW_SLC__1SDV_20150317T051938_20150317T052005_005067_0065D5_B405.json", Directory.GetCurrentDirectory());
IStacItem sentinel1Item_1 = (IStacItem)StacFactory.Load(s1_1_filepath);
IStacItem sentinel1Item_2 = (IStacItem)StacFactory.Load(s1_2_filepath);

// Load Satellite extension
SatStacExtension satExtension_1 = sentinel1Item_1.GetExtension<SatStacExtension>();
SatStacExtension satExtension_2 = sentinel1Item_2.GetExtension<SatStacExtension>();


Console.Out.WriteLine(string.Format("{0} track: {1} {2}", sentinel1Item_1.GetTitle(),
                                                            satExtension_1.RelativeOrbit, 
                                                            satExtension_1.OrbitState));
Console.Out.WriteLine(string.Format("{0} track: {1} {2}", sentinel1Item_2.GetTitle(),
                                                            satExtension_2.RelativeOrbit, 
                                                            satExtension_2.OrbitState));

Sentinel-1A SLC IW SDV T95 2015-03-05T05:19:37/2015-03-05T05:20:05 track: 95 descending
Sentinel-1A SLC IW SDV T95 2015-03-17T05:19:38/2015-03-17T05:20:05 track: 95 descending


#### 5.1 Load Orbital State Vectors

In the next code snippet, we will use the [Satellite Extension](https://github.com/radiantearth/stac-spec/tree/master/extensions/sat) plugin to perform an perpendicular baseline calculation between 2 Sentinel-1 SAR SLC items

![orbital elements](https://upload.wikimedia.org/wikipedia/commons/thumb/e/eb/Orbit1.svg/290px-Orbit1.svg.png "Orbital Elements")



In [6]:
satExtension_1.LoadOrbitStateVectors();
satExtension_2.LoadOrbitStateVectors();

satExtension_1.OrbitStateVectors

key,Time,Position,Velocity,Acceleration
2015-03-05 05:19:40Z,2015-03-05 05:19:40Z,"[ 5082939.418983107, 1595651.736905041, 4648033.558854568 ]","[ 5173.877932938749, -269.5432365485845, -5550.246440359806 ]",<null>
2015-03-05 05:19:45Z,2015-03-05 05:19:45Z,"[ 5110519.037654368, 1594174.328112772, 4618284.720257935 ]","[ 5143.120075407314, -283.1234366173459, -5578.216061064719 ]",<null>
2015-03-05 05:19:50Z,2015-03-05 05:19:50Z,"[ 5136162.388785399, 1592727.024757518, 4590328.590456041 ]","[ 5114.196195327355, -295.7938430458768, -5604.210861145829 ]",<null>
2015-03-05 05:19:55Z,2015-03-05 05:19:55Z,"[ 5164072.544175726, 1591069.832086559, 4559571.316048217 ]","[ 5082.352650799134, -309.6321010898558, -5632.491349430379 ]",<null>
2015-03-05 05:20:00Z,2015-03-05 05:20:00Z,"[ 5188609.760318574, 1589541.212127296, 4532241.960494119 ]","[ 5054.038535746737, -321.8414493178929, -5657.341909639592 ]",<null>
2015-03-05 05:20:05Z,2015-03-05 05:20:05Z,"[ 5213806.540800014, 1587900.544910171, 4503891.413309338 ]","[ 5024.649317879293, -334.4204559571785, -5682.851221401394 ]",<null>


#### 5.2 Calculate Perpendicular Baseline

Now we ask the Satellite Extension to calculate the Baseline for our 2 Satellite items

<img src="https://slideplayer.com/slide/8654127/26/images/39/Perpendicular+Baseline.jpg" width="400">


The purpose of this example is to show the capacity building of the plugin engine to provide with embedded functions out of the box when using STAC catalogs

In [7]:
var baseline = satExtension_1.CalculateBaseline(satExtension_2);
display(baseline);

Perpendicular,Parallel,Along
10.12702223151236,14.91706357528211,40.63389277031717


### Other features to come

* STAC API Client
* Importers from other spatio temporal domains (e.g. OGC O&M, Earth Observation profile…)

<h1 align="center">Thank you!</h1>
<img src="https://www.terradue.com/wp-content/uploads/2017/02/logo-03-01-1.svg" width="600" />

www.terradue.com<br/>
Emmanuel Mathot<br/>
emmanuel.mathot@terradue.com<br/>